# Week 1 Assignment 


In [50]:
import pandas as pd
import string
import numpy as np

In [8]:
products = pd.read_csv(r'C:\Users\Admin\Documents\Coursera Courses\Machine Learning - Classification\Week 1\amazon_baby.csv')

In [32]:
products.head()[:2]

,name,review,rating,Cleaned_Review
1,Planetwise Wipe Pouch,it came early and was not disappointed. i love...,5,it came early and was not disappointed. i love...
2,Annas Dream Full Quilt with 2 Shams,Very soft and comfortable and warmer than it l...,5,Very soft and comfortable and warmer than it l...


### Perform text cleaning

2. We start by removing punctuation, so that words "cake." and "cake!" are counted as the same word.

In [28]:

def removepunctiuation(lines):
#     cleanedline = [line.replace(string.punctuation,"") for line in list(lines)]
    return str(lines).replace(string.punctuation,"") 
products["Cleaned_Review"] = products["review"].apply(removepunctiuation)

In [29]:
products["review"].head()

0    These flannel wipes are OK, but in my opinion ...
1    it came early and was not disappointed. i love...
2    Very soft and comfortable and warmer than it l...
3    This is a product well worth the purchase.  I ...
4    All of my kids have cried non-stop when I trie...
Name: review, dtype: object

Aside. In this notebook, we remove all punctuation for the sake of simplicity. A smarter approach to punctuation would preserve phrases such as "I'd", "would've", "hadn't" and so forth. See this page for an example of smart handling of punctuation.

In [30]:
products = products.fillna({'review':''})

In [31]:
#3. We will ignore all reviews with rating = 3, since they tend to have a neutral sentiment. In SFrame, for instance,
products = products[products['rating'] != 3]

#4. Now, we will assign reviews with a rating of 4 or higher to be positive reviews, while the ones with rating of 2 or lower are negative. For the sentiment column, we use +1 for the positive class label and -1 for the negative class label. A good way is to create an anonymous function that converts a rating into a class label and then apply that function to every element in the rating column. In SFrame, you would use apply():



In [33]:
products["sentiments"] = products["rating"].apply(lambda rating: +1 if rating > 3 else -1)

Split into training and test sets

5. Let's perform a train/test split with 80% of the data in the training set and 20% of the data in the test set. If you are using SFrame, make sure to use seed=1 so that you get the same result as everyone else does. (This way, you will get the right numbers for the quiz.)

In [36]:
from sklearn.model_selection import train_test_split
train_data, test_data = train_test_split(products, test_size=.2)

6. We will now compute the word count for each word that appears in the reviews. A vector consisting of word counts is often referred to as bag-of-word features. Since most words occur in only a few reviews, word count vectors are sparse. For this reason, scikit-learn and many other tools use sparse matrices to store a collection of word count vectors. Refer to appropriate manuals to produce sparse word count vectors. General steps for extracting word count vectors are as follows:

Learn a vocabulary (set of all words) from the training data. Only the words that show up in the training data will be considered for feature extraction.
Compute the occurrences of the words in each review and collect them into a row vector.
Build a sparse matrix where each row is the word count vector for the corresponding review. Call this matrix train_matrix.
Using the same mapping between words and columns, convert the test data into a sparse matrix test_matrix.
The following cell uses CountVectorizer in scikit-learn. Notice the token_pattern argument in the constructor.

In [40]:
from sklearn.feature_extraction.text import CountVectorizer

vectorizer = CountVectorizer(token_pattern=r'\b\w+\b')
     # Use this token pattern to keep single-letter words
# First, learn vocabulary from the training data and assign columns to words
# Then convert the training data into a sparse matrix
train_matrix = vectorizer.fit_transform(train_data['Cleaned_Review'])
# Second, convert the test data into a sparse matrix, using the same word-column mapping
test_matrix = vectorizer.transform(test_data['Cleaned_Review'])

Keep in mind that the test data must be transformed in the same way as the training data.

Train a sentiment classifier with logistic regression

We will now use logistic regression to create a sentiment classifier on the training data.

7. Learn a logistic regression classifier using the training data. If you are using scikit-learn, you should create an instance of the LogisticRegression class and then call the method fit() to train the classifier. This model should use the sparse word count matrix (train_matrix) as features and the column sentiment of train_data as the target. Use the default values for other parameters. Call this model sentiment_model.

8. There should be over 100,000 coefficients in this sentiment_model. Recall from the lecture that positive weights w_j correspond to weights that cause positive sentiment, while negative weights correspond to negative sentiment. Calculate the number of positive (>= 0, which is actually nonnegative) coefficients.

In [42]:
from sklearn.linear_model import LogisticRegression
# Create classifiers
lr = LogisticRegression()
sentiment_model = lr.fit(train_matrix,train_data["sentiments"])

In [51]:
np.sum(sentiment_model.coef_ > 0)

39043

Making predictions with logistic regression

9. Now that a model is trained, we can make predictions on the test data. In this section, we will explore this in the context of 3 data points in the test data. Take the 11th, 12th, and 13th data points in the test data and save them to sample_test_data. The following cell extracts the three data points from the SFrame test_data and print their content:

In [58]:
sample_test_data = (test_data[10:13]).reset_index()
print(sample_test_data)

   index                                  name  \
0  55365       Bla bla kids Mini Rose Cat Doll   
1  69922  Boppy Protect Me Shopping Cart Liner   
2  49472          Graco imonitor Vibe 2 Parent   

                                              review  rating  \
0  We buy these dolls as gifts for the little one...       5   
1  I like this product, our son is a much happier...       4   
2  Graco imonitor Vibe. This monitor is highly un...       1   

                                      Cleaned_Review  sentiments  
0  We buy these dolls as gifts for the little one...           1  
1  I like this product, our son is a much happier...           1  
2  Graco imonitor Vibe. This monitor is highly un...          -1  


In [59]:
sample_test_data['review'][0]

'We buy these dolls as gifts for the little ones in our lives, and everyone adores them.  They are made very beautifully of high quality materials.  We machine was ours on delicate occasionally, and they hold up perfectly.  These are not disposable toys, more heirloom quality, which is how I justify the price.  Money well spent, I believe.'

That review seems pretty positive.

Now, let's see what the next row of the sample_test_data looks like. As we could guess from the rating (-1), the review is quite negative.

In [60]:
sample_test_data['review'][1]

'I like this product, our son is a much happier shopper using this than a plain shopping cart.  We can put some toys beside him, or on the loops. It has convenient pockets on the back for snacks, more toys, etc.  My only problem with it is that you still have to use the nasty belt from the shopping cart.'

10. We will now make a class prediction for the sample_test_data. The sentiment_model should predict +1 if the sentiment is positive and -1 if the sentiment is negative. Recall from the lecture that the score (sometimes called margin) for the logistic regression model is defined as:

scorei=w⊺h(xi)
where h(xi) represents the features for data point i. We will write some code to obtain the scores. For each row, the score (or margin) is a number in the range (-inf, inf). Use a pre-built function in your tool to calculate the score of each data point in sample_test_data. In scikit-learn, you can call the decision_function() function.

Hint: You'd probably need to convert sample_test_data into the sparse matrix format first.

In [63]:
sample_test_matrix = vectorizer.transform(sample_test_data['Cleaned_Review'])
scores = sentiment_model.decision_function(sample_test_matrix)
print(scores)

[ 8.34316727  5.45180159 -8.54164803]


In [69]:
def prediction(scores):
    x = [(lambda score: +1 if score > 0 else -1)(score) for score in scores]
    return x
prediction(scores)

[1, 1, -1]

In [65]:
sentiment_model.predict(sample_test_matrix)

array([ 1,  1, -1], dtype=int64)

Probability Predictions

12. Recall from the lectures that we can also calculate the probability predictions from the scores using:

P(yi=+1|xi,w)=11+exp(−w⊺h(xi))
Using the scores calculated previously, write code to calculate the probability that a sentiment is positive using the above formula. For each row, the probabilities should be a number in the range [0, 1].

In [83]:
def probability_predict(scores):
    x = [[1-(1/(1+np.exp(-1 * score))),1/(1+np.exp(-1 * score))] for score in scores]
    return x
probability_predict(scores)

[[0.00023796064431469421, 0.99976203935568531],
 [0.0042702581651978821, 0.99572974183480212],
 [0.99980486973110994, 0.00019513026889002202]]

In [84]:
sentiment_model.predict_proba(sample_test_matrix)

array([[  2.37960644e-04,   9.99762039e-01],
       [  4.27025817e-03,   9.95729742e-01],
       [  9.99804870e-01,   1.95130269e-04]])

Find the most positive (and negative) review

13. We now turn to examining the full test dataset, test_data, and use sklearn.linear_model.LogisticRegression to form predictions on all of the test data points.

Using the sentiment_model, find the 20 reviews in the entire test_data with the highest probability of being classified as a positive review. We refer to these as the "most positive reviews."

To calculate these top-20 reviews, use the following steps:

Make probability predictions on test_data using the sentiment_model.
Sort the data according to those predictions and pick the top 20.

In [85]:
test_matrix = vectorizer.transform(test_data['Cleaned_Review'])

In [87]:
test_probabilities = sentiment_model.predict_proba(test_matrix)
test_probabilities

array([[  2.10843513e-04,   9.99789156e-01],
       [  3.56574657e-02,   9.64342534e-01],
       [  2.36358693e-03,   9.97636413e-01],
       ..., 
       [  1.30484173e-04,   9.99869516e-01],
       [  4.70624329e-01,   5.29375671e-01],
       [  1.17156158e-02,   9.88284384e-01]])

14. Now, let us repeat this exercise to find the "most negative reviews." Use the prediction probabilities to find the 20 reviews in the test_data with the lowest probability of being classified as a positive review. Repeat the same steps above but make sure you sort in the opposite order.

In [120]:
np.argsort(test_probabilities[:,0:1])

array([[0],
       [0],
       [0],
       ..., 
       [0],
       [0],
       [0]], dtype=int64)

In [121]:
######Testing
arr = [[5,3,7,2],[6,34,46,344],[545,32,5,22]]

In [150]:
######Testing
np.argsort(arr,axis=1)[:,1:-1]

array([[1, 0],
       [1, 2],
       [3, 1]], dtype=int64)

In [148]:
######Testing
np.argsort(arr,axis=1)

array([[3, 1, 0, 2],
       [0, 1, 2, 3],
       [2, 3, 1, 0]], dtype=int64)

In [162]:
test_data.assign(Predicted = pd.Series(sentiment_model.predict(test_matrix)),index=test_data.index)
# sentiment_model.predict(test_matrix)

,name,review,rating,Cleaned_Review,sentiments,sentiments_Predicted,Predicted,index
86175,Kinderville Little Bites Cups Set of Four,These cups are fantastic for teaching your tod...,4,These cups are fantastic for teaching your tod...,1,1,NaN,86175
17803,Westminster Hand Boilers (Colors May Vary),This is such a neat little toy. And for the pr...,5,This is such a neat little toy. And for the pr...,1,1,1.0,17803
154859,"Hevea Pacifier Clip, Pink",Works great. I use it to keep baby's teether a...,5,Works great. I use it to keep baby's teether a...,1,1,NaN,154859
60857,Lifefactory Glass Baby Bottle with Silicone Sl...,Bought as baby gift. Parents loved it. They ar...,5,Bought as baby gift. Parents loved it. They ar...,1,1,NaN,60857
142452,"Graco Pack 'n Play Element Playard, Metropolis",Works for babies and toddlers and allows you t...,5,Works for babies and toddlers and allows you t...,1,1,NaN,142452
104649,"Kathe Kruse Baby Doll, Elli",As long as she likes I'm happy. I like the fac...,4,As long as she likes I'm happy. I like the fac...,1,1,NaN,104649
173838,Bundle Monster 12pc Multicolor Satin Peony Flo...,"Loved the assorted colors, few of the petals ...",4,"Loved the assorted colors, few of the petals ...",1,-1,NaN,173838
46138,Kids Preferred Amazing Baby Feel and Learn - S...,"Our baby loves it - different textures, pictur...",5,"Our baby loves it - different textures, pictur...",1,1,NaN,46138
97554,BabyKicks Natural &amp; Organic 10 Pack Baby W...,We use these every night as our washcloths. An...,5,We use these every night as our washcloths. An...,1,1,NaN,97554
101294,OXO Tot Bottle Brush with Nipple Cleaner and S...,"Easy to use, does the job, like the stand. Pre...",4,"Easy to use, does the job, like the stand. Pre...",1,1,NaN,101294


In [165]:
test_data

,name,review,rating,Cleaned_Review,sentiments,sentiments_Predicted
86175,Kinderville Little Bites Cups Set of Four,These cups are fantastic for teaching your tod...,4,These cups are fantastic for teaching your tod...,1,1
17803,Westminster Hand Boilers (Colors May Vary),This is such a neat little toy. And for the pr...,5,This is such a neat little toy. And for the pr...,1,1
154859,"Hevea Pacifier Clip, Pink",Works great. I use it to keep baby's teether a...,5,Works great. I use it to keep baby's teether a...,1,1
60857,Lifefactory Glass Baby Bottle with Silicone Sl...,Bought as baby gift. Parents loved it. They ar...,5,Bought as baby gift. Parents loved it. They ar...,1,1
142452,"Graco Pack 'n Play Element Playard, Metropolis",Works for babies and toddlers and allows you t...,5,Works for babies and toddlers and allows you t...,1,1
104649,"Kathe Kruse Baby Doll, Elli",As long as she likes I'm happy. I like the fac...,4,As long as she likes I'm happy. I like the fac...,1,1
173838,Bundle Monster 12pc Multicolor Satin Peony Flo...,"Loved the assorted colors, few of the petals ...",4,"Loved the assorted colors, few of the petals ...",1,-1
46138,Kids Preferred Amazing Baby Feel and Learn - S...,"Our baby loves it - different textures, pictur...",5,"Our baby loves it - different textures, pictur...",1,1
97554,BabyKicks Natural &amp; Organic 10 Pack Baby W...,We use these every night as our washcloths. An...,5,We use these every night as our washcloths. An...,1,1
101294,OXO Tot Bottle Brush with Nipple Cleaner and S...,"Easy to use, does the job, like the stand. Pre...",4,"Easy to use, does the job, like the stand. Pre...",1,1


In [171]:
test_data["correct_incorrect"] = test_data[["sentiments","sentiments_Predicted"]].apply(lambda x: 1 if x["sentiments"]==x["sentiments_Predicted"] else 0, axis=1)

C:\Users\Admin\Anaconda3\lib\site-packages\ipykernel\__main__.py:1: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: http://pandas.pydata.org/pandas-docs/stable/indexing.html#indexing-view-versus-copy
  if __name__ == '__main__':


In [172]:
test_data

,name,review,rating,Cleaned_Review,sentiments,sentiments_Predicted,correct_incorrect
86175,Kinderville Little Bites Cups Set of Four,These cups are fantastic for teaching your tod...,4,These cups are fantastic for teaching your tod...,1,1,1
17803,Westminster Hand Boilers (Colors May Vary),This is such a neat little toy. And for the pr...,5,This is such a neat little toy. And for the pr...,1,1,1
154859,"Hevea Pacifier Clip, Pink",Works great. I use it to keep baby's teether a...,5,Works great. I use it to keep baby's teether a...,1,1,1
60857,Lifefactory Glass Baby Bottle with Silicone Sl...,Bought as baby gift. Parents loved it. They ar...,5,Bought as baby gift. Parents loved it. They ar...,1,1,1
142452,"Graco Pack 'n Play Element Playard, Metropolis",Works for babies and toddlers and allows you t...,5,Works for babies and toddlers and allows you t...,1,1,1
104649,"Kathe Kruse Baby Doll, Elli",As long as she likes I'm happy. I like the fac...,4,As long as she likes I'm happy. I like the fac...,1,1,1
173838,Bundle Monster 12pc Multicolor Satin Peony Flo...,"Loved the assorted colors, few of the petals ...",4,"Loved the assorted colors, few of the petals ...",1,-1,0
46138,Kids Preferred Amazing Baby Feel and Learn - S...,"Our baby loves it - different textures, pictur...",5,"Our baby loves it - different textures, pictur...",1,1,1
97554,BabyKicks Natural &amp; Organic 10 Pack Baby W...,We use these every night as our washcloths. An...,5,We use these every night as our washcloths. An...,1,1,1
101294,OXO Tot Bottle Brush with Nipple Cleaner and S...,"Easy to use, does the job, like the stand. Pre...",4,"Easy to use, does the job, like the stand. Pre...",1,1,1


In [175]:
sum(test_data["correct_incorrect"])/len(test_data)

0.9321159785313784

Learn another classifier with fewer words

16. There were a lot of words in the model we trained above. We will now train a simpler logistic regression model using only a subet of words that occur in the reviews. For this assignment, we selected 20 words to work with. These are:

In [176]:
significant_words = ['love', 'great', 'easy', 'old', 'little', 'perfect', 'loves', 
      'well', 'able', 'car', 'broke', 'less', 'even', 'waste', 'disappointed', 
      'work', 'product', 'money', 'would', 'return']

In [178]:
vectorizer_word_subset = CountVectorizer(vocabulary=significant_words) # limit to 20 words
train_matrix_word_subset = vectorizer_word_subset.fit_transform(train_data['Cleaned_Review'])
test_matrix_word_subset = vectorizer_word_subset.transform(test_data['Cleaned_Review'])

In [179]:
simple_model = lr.fit(train_matrix_word_subset,train_data["sentiments"])

In [180]:
simple_model_coef_table = pd.DataFrame({'word':significant_words,
                                         'coefficient':simple_model.coef_.flatten()})

In [182]:
simple_model_coef_table.sort_values(by = ["coefficient"],ascending=[0])

,coefficient,word
6,1.712034,loves
5,1.484256,perfect
0,1.371091,love
2,1.178452,easy
1,0.941975,great
7,0.525123,well
4,0.493150,little
8,0.180648,able
3,0.094545,old
9,0.064318,car
